In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('web-traffic-time-series-forecasting/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_df = pd.read_csv("C:/Users/Lenovo/Desktop/Web project/train_2.csv")
train_df = train_df.dropna()

In [4]:

# Extract 'Page' names and remove the specified part
page_names = train_df['Page'].str.replace('.wikipedia.org_all-access_spider', '')

# Create a new DataFrame with the modified 'Page' names
result_df = pd.DataFrame({'Modified_Page': page_names})

# Save the new DataFrame to a CSV file
result_df.to_csv('C:/Users/Lenovo/Desktop/Web project/page_names.csv', index=False)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20160\1795081665.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  page_names = train_df['Page'].str.replace('.wikipedia.org_all-access_spider', '')


In [5]:
train_df = pd.melt(train_df[list(train_df.columns[-100:]) + ['Page']], id_vars='Page', var_name='date', value_name='Visits')

train_df['date'] = train_df['date'].astype('datetime64[ns]')

train_df['weekday'] = train_df['date'].apply(lambda x: x.weekday())

In [6]:
train_df

,Page,date,Visits,weekday
0,2NE1_zh.wikipedia.org_all-access_spider,2017-06-03,30.0,5
1,2PM_zh.wikipedia.org_all-access_spider,2017-06-03,31.0,5
2,3C_zh.wikipedia.org_all-access_spider,2017-06-03,4.0,5
3,4minute_zh.wikipedia.org_all-access_spider,2017-06-03,14.0,5
4,5566_zh.wikipedia.org_all-access_spider,2017-06-03,34.0,5
...,...,...,...,...
11508395,Chichén_Itzá_es.wikipedia.org_all-access_spider,2017-09-10,5.0,6
11508396,Fecundación_es.wikipedia.org_all-access_spider,2017-09-10,39.0,6
11508397,Gran_Hermano_VIP_(España)_es.wikipedia.org_all...,2017-09-10,7.0,6
11508398,Modelo_atómico_de_Thomson_es.wikipedia.org_all...,2017-09-10,26.0,6


In [ ]:
# Get user input for the date
user_date = input("Enter the date (in the format 'YYYY-MM-DD'): ")

# Filter the DataFrame for the specified date and visits greater than 0
selected_date_df = train_df[(train_df['date'] == user_date) & (train_df['Visits'] > 0)]

# Find the row with the minimum visits and the row with the maximum visits
min_visits_row = selected_date_df.loc[selected_date_df['Visits'].idxmin()]
max_visits_row = selected_date_df.loc[selected_date_df['Visits'].idxmax()]

# Display the result
print(f"\nFor the date {user_date}:")
print("Least Visited Page:")
print(f"Page: {min_visits_row['Page']}, Visits: {min_visits_row['Visits']}")

print("\nMost Visited Page:")
print(f"Page: {max_visits_row['Page']}, Visits: {max_visits_row['Visits']}")


In [ ]:
train_df[::-1]

In [ ]:
df = train_df.groupby(['date']).agg({'Visits':'sum'}).rename(columns={'Visits':'visit'})

In [ ]:
df

## ACF, PACF Plot

In [ ]:
!pip install pmdarima
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf,  plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima

In [ ]:
from sklearn.model_selection import train_test_split 
train_df, test_df = train_test_split(df, test_size = 0.2, shuffle=False)

In [ ]:
test_df.to_csv("testdf.csv",index=False)

In [ ]:
train_df.to_csv("traindf.csv",index=False)

In [ ]:
decomposition = sm.tsa.seasonal_decompose(df, model='multiplicative')

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
fig, axes = plt.subplots(4, 1, figsize=(20, 20))
axes[0].plot(df, label='page visits')
axes[0].grid()
axes[0].legend(loc='upper right')
axes[1].plot(trend, label='Trend')
axes[1].grid()
axes[1].legend(loc='upper right')
axes[2].plot(seasonal, label='Seasonal')
axes[2].grid()
axes[2].legend(loc='upper right')
axes[3].plot(residual, label='Residuals')
axes[3].grid()
axes[3].legend(loc='upper right')


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle('Raw Data')

sm.graphics.tsa.plot_acf(train_df.values.squeeze(), lags=30, ax=ax[0])
sm.graphics.tsa.plot_pacf(train_df.values.squeeze(), lags=30, ax=ax[1])


## Autocorrelation
- slowly decrease shape of sin graph
- it's typically seasonal non-stationary timeseries's autocorrelation
- so we need to diff the data

In [ ]:
# 1 differencing
diff_train_df = train_df.copy()
diff_train_df = diff_train_df.visit.diff()
diff_train_df = diff_train_df.dropna()
diff_train_df.columns =['visit']
diff_train_df

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8))


axes[0].plot(train_df.visit, label='visit')
axes[1].plot(diff_train_df, 'orange', label='diff_1 visit')

axes[0].grid()
axes[1].grid()
axes[0].legend(loc='upper left')
axes[1].legend(loc='upper left')

- diff_1 visit data look like more stationary than origin visit data
- let's plot acf, pacf chart of diff_1 visit data

In [ ]:
def test_stationary(df):
    fig, axes = plt.subplots(2, 1, figsize=(10,5))
    rolling_mean = df['visit'].rolling(7).mean()
    rolling_std = df['visit'].rolling(7).std()
    
    axes[0].plot(df['visit'], color='blue', label= 'origin')
    axes[0].plot(rolling_mean, color='red', label='Rolling Mean')
    axes[1].plot(rolling_std, color='black', label='Rolling Std')
    axes[0].grid(True)
    axes[1].grid(True)
    
    dftest = sm.tsa.adfuller(df['visit'], autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', 'Lags Used', '# of observation used'])
    for k, v in dftest[4].items():
        dfoutput[f'Critical Value {k}'] = v
    print(dfoutput)
    
    
test_stationary(train_df)

In [ ]:
test_df.index

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
# fig.suptitle('Diff 1 Visit')
sm.graphics.tsa.plot_acf(diff_train_df.values.squeeze(), lags=30, ax=ax[0])
sm.graphics.tsa.plot_pacf(diff_train_df.values.squeeze(), lags=30, ax=ax[1])
# statsmodel pacf plot function has issue plot chart twice  

## How to set p, d, q ARIMA Model Value in this case, using Graphical Method
- In ACF Chart, values are die out shape of sine-function graph => Choose AR Model
- In partial ACF chart, cut off after lag of p, in this case 3 is laf value make cuf off below 0 => set p = 3 
- we use diff_1 visit value, so d = 1
- we do not use MA model in this case (MA Model is chosen when ACF Chart look like cut off after `q` lag)
    - so q = 0
- so, we use ARIMA(3, 1, 0) Model


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(train_df.values, order=(2, 1, 2))
model_fit = model.fit()
model_fit.summary()

- ARIMA(3, 1, 0) has very high p-value
- let's diagnose arima pdq value

In [ ]:
import itertools
p = range(0, 3)
d = range(1, 3)
q = range(0, 3)

pdq = list(itertools.product(p, d, q))

aic = []
for i in pdq:
    model = ARIMA(train_df.values, order=(i))
    model_fit = model.fit()
    print(f'ARIMA({i}) AIC : {round(model_fit.aic,2)}')
    aic.append(round(model_fit.aic, 2))

In [ ]:
# ARIMA((2, 1, 2)) is best aic

model = ARIMA(train_df.values, order=(2, 2, 1))
model_fit = model.fit()
model_fit.summary()

- ARIMA(2, 1, 2) model has good p-value
- let's predict test_df's visit data

In [ ]:
prediction = model_fit.forecast(len(test_df))
prediction_value = prediction
prediction_index = list(test_df.index)


In [ ]:
fig, axes = plt.subplots(figsize=(20, 14))


axes.plot(train_df)
axes.plot(prediction_index, prediction_value,color='r', label='Prediction')
axes.plot(test_df, linestyle='--', color='orange', label='Real Visit Data')
axes.legend(loc='upper left')
axes.grid()

- Let's Make bigger P, Q Value
- ARIMA(7, 1, 7)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Assuming your data frame has a datetime index
start_date = input("Enter start date (yyyy-mm-dd): ")
end_date = input("Enter end date (yyyy-mm-dd): ")

# Convert input strings to datetime objects
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

# Filter data based on user input
train_df_filtered = train_df[(train_df.index >= start_date) & (train_df.index <= end_date)]

# Fit ARIMA model
model = ARIMA(train_df_filtered, order=(7, 1, 7))
model_fit = model.fit()

# Plotting
fig, axes = plt.subplots(figsize=(15, 8))
axes.plot(train_df_filtered, label='page visit')
axes.plot(model_fit.fittedvalues[2:], label='predict')
axes.plot(pd.DataFrame(model_fit.forecast(steps=len(test_df))), label='predict', color='red', linestyle='--')

axes.set_xlim([start_date, end_date])

plt.xlabel("Date")
axes.grid()
axes.legend()

plt.show()


In [ ]:
len(test_df)

In [ ]:
import pickle

with open("webtraffic.pkl","wb") as file:
    pickle.dump(model_fit,file)

In [ ]:
print(test_df.index)